In [3]:
import pandas as pd
from ortools.sat.python import cp_model

data = pd.read_csv('TRIAL_DATA.csv')
new_dataset = data.dropna().copy()
print('Dataset: ', new_dataset)

# Priority mapping
priority_mapping = {
    "Locked In activity": 1,
    "Scheduled activity": 2,
    "Recurring high activity": 3,
    "Recurring medium activity": 4,
    "Recurring low activity": 5,
    "To-do activity": 6,
    "Bucket list activity": 7,
}

# Map categories to priorities
new_dataset['priority'] = new_dataset['category'].map(priority_mapping)
new_dataset.drop(columns=['category'], inplace=True)
print('Dataset with priority: ', new_dataset)

def time_to_minutes(time_string):
    if pd.isnull(time_string):
        return None
    time = pd.to_datetime(time_string, format= '%H:%M').time()
    return time.hour * 60 + time.minute
    
new_dataset['preferred_time_start'] = new_dataset['preferred_time_start'].apply(time_to_minutes)
new_dataset['preferred_time_end'] = new_dataset['preferred_time_end'].apply(time_to_minutes)
print('Dataset with priority and Minutes: ', new_dataset)

new_dataset.sort_values(by='priority', inplace=True)
print('Dataset with priority listing and Minutes: ', new_dataset)

model = cp_model.CpModel()

start_vars = {}
end_vars = {}
intervals = []
new_dataset['estimated_durations'] = new_dataset['estimated_durations'].fillna(60)
durations = new_dataset['estimated_durations'].astype(int)

def calculate_penalty(start_time, preferred_start):
    deviation_penalty = model.NewIntVar(0, 1440, f"penalty_{i}");
    model.AddAbsEquality(deviation_penalty, start_time - preferred_start)
    print('Deviation Penalty: ', deviation_penalty)
    return deviation_penalty

for i, row in new_dataset.iterrows():
    preferred_start = int(row['preferred_time_start'])
    preferred_end = int(row['preferred_time_end'])
    duration = int(durations[i])

    start_vars[i] = model.NewIntVar(preferred_start, preferred_end - duration, f"start_{i}")
    end_vars[i] = model.NewIntVar(preferred_start + duration, preferred_end, f"end_{i}")

    model.Add(end_vars[i] == start_vars[i] + duration)

    deviation_penalty = calculate_penalty(start_vars[i], preferred_start)

    interval = model.NewIntervalVar(start_vars[i], duration, end_vars[i], f"interval_{i}")
    intervals.append(interval)

model.AddNoOverlap(intervals)

model.minimize(
    sum(calculate_penalty(start_vars[i], int(new_dataset.iloc[i]['preferred_time_start'])) * new_dataset.iloc[i]['priority'] for i in range (len(new_dataset)))
)

solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 60

status = solver.solve(model)
print('Status: ', status)
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("Schedule: ")
    for i in range(len(new_dataset)):
        start_time = solver.Value(start_vars[i])
        start_hour, start_minute = divmod(start_time, 60)
        end_time = solver.Value(end_vars[i])
        end_hour, end_minute = divmod(end_time, 60)
        print(f"Activity {new_dataset.iloc[i]['activity_name']} starts at {start_hour:00}:{start_minute:00} and ends at {end_hour:00}:{end_minute:00}")
else:
    print("No feasible schedule found")

Dataset:     activity_name            category involved_person  estimated_durations  \
0     activity_1  Locked In activity             Sam                 59.0   
1     activity_2  Locked In activity             Sam                 59.0   
2     activity_3  Locked In activity             Sam                 59.0   
3     activity_4  Locked In activity             Sam                 59.0   
4     activity_5  Locked In activity             Sam                 59.0   
5     activity_6  Locked In activity             Sam                 59.0   
6     activity_7  Locked In activity             Sam                 59.0   
7     activity_8  Locked In activity             Sam                 59.0   
8     activity_9  Locked In activity             Sam                 59.0   
9    activity_10  Locked In activity             Sam                 59.0   
10   activity_11  Locked In activity             Sam                 59.0   
11   activity_12  Locked In activity             Sam              